# 1. Importar las bibliotecas a utilizar:

In [ ]:
import cobra  
import escher
%matplotlib inline
pd.options.display.max_rows = 30
from cameo.flux_analysis import phenotypic_phase_plane
from cameo.visualization.plotting.with_plotly import PlotlyPlotter
from cobra.flux_analysis import flux_variability_analysis
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
from IPython.display import display
import gurobipy
from cobra.io import read_sbml_model
from cameo.strain_design.heuristic.evolutionary_based import OptGene
plotter = PlotlyPlotter()

# 2. Cargar modelos

In [ ]:
modelCore = read_sbml_model('./models/modelCore_clean.xml')
modelCoreA = read_sbml_model('./models/modelCoreA_clean.xml')
modeliML = read_sbml_model('./models/modeliML_clean.xml')
modeliMLA = read_sbml_model('./models/modeliMLA_clean.xml')

Puedo ver un resumen de los modelos y el resultado de FBA utilizando la función summary():

In [ ]:
display(modelCore.summary())
display(modelCoreA.summary())
display(modeliML.summary())
display(modeliMLA.summary())

Defino una función que calcula el FBA de los modelos, la tasa de crecimiento, el flujo del compuesto, el flujo de glucos y el rendimiento del compuesto:

In [ ]:
def fba(model, obj, comp):
    solution = model.optimize()
    growthRate = solution.fluxes[obj]
    compFlux = solution.fluxes[comp] 
    gluFlux = solution.fluxes['EX_glc__D_e']
    yieldComp = compFlux/(-1*gluFlux)
    return growthRate, compFlux, yieldComp

Defino funcion que calcula FVA para revisar el flujo máximo y mínimo posible de las reacciones que forman parte de la ruta sintetizadora del compuesto de interés:

In [ ]:
def fvaReactions(model, rx, FoP):
    reactions = []
    for x in rx:
        reactions.append(getattr(model.reactions, x))
    if(FoP == None):
        fva = flux_variability_analysis(model, reaction_list = reactions)
        return fva
    else:
        fva = flux_variability_analysis(model, reaction_list = reactions, fraction_of_optimum=FoP)
        return fva

Obtengo FBA:

In [ ]:
# fba(modelCore,'BIOMASS_Ecoli_core_w_GAM' , 'EX_btd_RR_e')
fba(modelCoreA,'BIOMASS_Ecoli_core_w_GAM' , 'EX_btd_RR_e')
fba(modeliML,'BIOMASS_Ec_iML1515_core_75p37M' , 'EX_btd_RR_e')
fba(modeliMLA,'BIOMASS_Ec_iML1515_core_75p37M' , 'EX_btd_RR_e')

Defino una lista con las reacciones que forman parte de la síntesis del compuesto:

In [ ]:
rx_bd = ['ACLS', 'ACLDC', 'BTD_RR', 'BTDt_RR', 'EX_btd_RR_e']

Calculo FVA de esas reacciones:

In [ ]:
fvaCore = fvaReactions(modelCore, rx_bd, 0.9)
fvaCoreA = fvaReactions(modelCoreA, rx_bd, 0.9)
fvaiML = fvaReactions(modeliML, rx_bd, 0.9)
fvaiMLA = fvaReactions(modeliMLA, rx_bd, 0.9)

Defino una función que modifica el límite inferior de las reacciones:

In [ ]:
def lower_bound(model, fva, reaction):
    print('##########################################')
    for x in reaction:
        rx = getattr(model.reactions, x)
        maxVal = getattr(fva.maximum, x)
        print(x, maxVal)
        rx.lower_bound = maxVal

In [ ]:
lower_bound(modelCore, fvaCore, rx_bd)
lower_bound(modelCoreA, fvaCoreA, rx_bd)
lower_bound(modeliML, fvaiML, rx_bd)
lower_bound(modeliMLA, fvaiMLA, rx_bd)

Defino una función que calcula el análisis de plano de fase fenotípica:

In [ ]:
def ppp(model, biomass, target):
    pppsol = phenotypic_phase_plane(model, variables=[biomass], objective=target)
    return pppsol

In [ ]:
pppCore = ppp(modelCore, modelCore.reactions.BIOMASS_Ecoli_core_w_GAM, modelCore.reactions.EX_btd_RR_e)
display(pppCore.data_frame)
display(pppCore.plot(plotter))

Exporto los modelos analizados:

In [ ]:
from cobra.io import save_json_model
save_json_model(modelCoreA, "models/modelCoreA_bound.json")
save_json_model(modelCore, "models/modelCore_bound.json")
save_json_model(modeliML, "models/modeliML_bound.json")
save_json_model(modeliMLA, "models/modeliMLA_bound.json")